### Retrival Augmentented Generation

Vector embeddings

In [ ]:
pip install langchain-cohere
#pip install spacy

In [ ]:
# now you need to run this in a terminal window
# python -m spacy download en_core_web_lg

In [2]:
import boto3
import json
import numpy as np
from langchain.embeddings import BedrockEmbeddings
from langchain.embeddings import SpacyEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_cohere import CohereEmbeddings

# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

In [ ]:
def generate_spacy_vector_embedding(text):
    embeddings = SpacyEmbeddings(model="en_core_web_lg")
    embedded_texts = embeddings.embed_query(text)
    return(embedded_texts)

In [ ]:
def generate_huggingface_vector_embedding(text):
    embeddings = HuggingFaceEmbeddings(model_name="bert-base-uncased")
    embedded_texts = embeddings.embed_query(text_data)
    return(embedded_texts)

In [8]:
def generate_cohere_vector_embedding(text_data):
    input_type = "clustering"
    truncate = "NONE" # optional
    model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"
    
    # Create the JSON payload for the request
    json_params = {
            'texts': texts,
            'truncate': truncate, 
            "input_type": input_type
        }
    json_body = json.dumps(json_params)
    params = {'body': json_body, 'modelId': model_id,}
    
    # Invoke the model and print the response
    result = aws_client.invoke_model(**params)
    response = json.loads(result['body'].read().decode())
    return(response)


In [ ]:
# Let's generate a dense vector using Amazon Titan with LangChain
def generate_titan_vector_embedding(text):
    #create an Amazon Titan Text Embeddings client
    embeddings_client = BedrockEmbeddings(region_name="us-west-2") 

    #Invoke the model
    embedding = embeddings_client.embed_query(text)
    return(np.array(embedding))



In [14]:
# king_vector = generate_vector_embedding("King")
# queen_vector = generate_vector_embedding("Queen")
# man_vector = generate_vector_embedding("man")
# woman_vector = generate_vector_embedding("woman")
# Input cohere for embedding 
texts = ["King",
         "Queen",
         "man",
         "woman",
         ]
cohere_emdeddings = generate_cohere_vector_embedding(texts)

king_vector = np.array(cohere_emdeddings['embeddings'][0])
queen_vector = np.array(cohere_emdeddings['embeddings'][1])
man_vector = np.array(cohere_emdeddings['embeddings'][2])
woman_vector = np.array(cohere_emdeddings['embeddings'][3])
print(king_vector[:5])

[ 0.02429199 -0.02442932  0.02479553 -0.04058838 -0.06268311]


In [12]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity


similarity = cosine_similarity(king_vector, queen_vector)
print(f"Cosine Similarity: {similarity:.4f}")

Cosine Similarity: 0.7355


In [15]:
calculated_queen_vector = king_vector - man_vector + woman_vector

similarity = cosine_similarity(calculated_queen_vector, queen_vector)
print(f"Cosine Similarity: {similarity:.4f}")


Cosine Similarity: 0.7239
